# Chatbots

Following: https://python.langchain.com/v0.2/docs/tutorials/chatbot/

In [2]:
from langchain_community.chat_models import ChatOllama

In [3]:
model = ChatOllama(model="llama3")

## General Chat

In [4]:
from langchain_core.messages import HumanMessage

In [5]:
model.invoke([HumanMessage(content="Hello, I'm Oliver")])

AIMessage(content="Nice to meet you, Oliver! How's your day going so far?", response_metadata={'model': 'llama3', 'created_at': '2024-06-06T17:49:47.1032911Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 9461056400, 'load_duration': 9138393300, 'prompt_eval_count': 14, 'prompt_eval_duration': 61393000, 'eval_count': 16, 'eval_duration': 256060000}, id='run-9e629072-32ff-47f9-81ed-90872a1332db-0')

In [6]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='I just learned it from you - your name is Bob!', response_metadata={'model': 'llama3', 'created_at': '2024-06-06T17:49:49.4643185Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 321315700, 'load_duration': 2070600, 'prompt_eval_count': 35, 'prompt_eval_duration': 111982000, 'eval_count': 13, 'eval_duration': 205069000}, id='run-36eb5aa7-333e-4093-8267-3d84ef6b3d4a-0')

## Message history

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables.config import RunnableConfig

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [8]:
config = RunnableConfig(configurable={"session_id": "1337"})

In [9]:
response = with_message_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)

In [10]:
response.content

'Nice to meet you, Oliver! How are you today?'

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")], config=config)

In [12]:
response.content

'Your name is Oliver!'

# Prompt templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [14]:
response = chain.invoke({"messages": [HumanMessage(content="Hello, I'm Oliver")]})
response.content

"Nice to meet you, Oliver! How can I help you today? Do you have any specific questions or topics you'd like to discuss?"

And now with history:

In [15]:
with_history = RunnableWithMessageHistory(chain, get_session_history)

In [16]:
config = RunnableConfig(configurable={"session_id": "1338"})

In [17]:
response = with_history.invoke(
    [HumanMessage(content="Hello, I'm Oliver")], config=config)
response.content

"Nice to meet you, Oliver! How can I help or assist you today? Do you have any specific questions or topics you'd like to discuss? I'm here to listen and provide helpful information!"

In [18]:
response = with_message_history.invoke(
    [HumanMessage(content="Write a haiku that greets me by my name")],
    config=config,
)

print(response.content)

Oliver's bright dawn
Golden light on morning face
Welcome, dear one


## More template variables

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [24]:
response = chain.invoke(
    {"messages": [HumanMessage(content="hi! I'm bob")], "language": "Spanish"}
)

response.content

'Hola Bob! ¡Bienvenido! ¿En qué puedo ayudarte hoy? (Hi Bob! Welcome! How can I help you today?)'

In [33]:
with_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [34]:
config = {"configurable": {"session_id": "abc11"}}

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="hi! I'm todd")], "language": "Spanish"},
    config=config,
)

response.content

"¡Hola Todd! ¡Vamos a empezar de nuevo! ¿En qué puedo ayudarte hoy? Tengo conocimientos en various áreas, desde traducciones hasta respuestas para preguntas curiosas. ¡Soy tu asistente personal en español! (Hi Todd! Let's start again! What can I help you with today? I have knowledge in various areas, from translations to answers for curious questions. I'm your personal assistant in Spanish!)"

## Restrict message size

In [36]:
from langchain_core.runnables import RunnablePassthrough


def filter_messages(messages, k=10):
    return messages[-k:]


chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

In [37]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

In [38]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "English",
    }
)
response.content

"I don't think we've established a name for you yet. Would you like to choose one or shall I give you a nickname?"

In [39]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what flavor of ice cream do i like?")],
        "language": "English",
    }
)
response.content

'vanilla!'